# [Assignment 4](https://ovgu-ailab.github.io/idl2023/assignment4.html)

collaborative work from Adrian Bremer & Philipp Reinig

## Setup



In [7]:
import tensorflow as tf
import numpy as np
from datetime import datetime

## Graph based execution

Trying out tf.function on MLP from assignment 2

In [8]:
class MLP:
    
    def __init__(self, layer_sizes, init_val):
        if len(layer_sizes) <= 1:
            raise AssertionError("There has to be at least 2 layers. 'layer_sizes' was {}".format(layer_sizes))
        
        self.weights = [tf.Variable(np.random.uniform(low=-init_val, high=init_val, size=(layer_sizes[i], layer_sizes[i+1])).astype(np.float32)) for i in range(len(layer_sizes)-1)]
        self.biases = [tf.Variable(np.zeros(layer_sizes[i]).astype(np.float32)) for i in range(1,len(layer_sizes))]
    
    def model(self, input_batch):
        """returns the logits for a given input batch"""
        
        n = len(self.biases)
        if n == 1:
            return tf.matmul(input_batch, self.weights[0]) + self.biases[0]
        
        next_input = input_batch
        for i in range(n-1):
            next_input = tf.nn.leaky_relu(tf.matmul(next_input, self.weights[i]) + self.biases[i])
        
        return tf.matmul(next_input, self.weights[n-1]) + self.biases[n-1]
    
    def train(self, imgs, lbls, learning_rate):
        """returns the cross entropy error"""
        n = len(self.biases)
      
        with tf.GradientTape() as tape:
            logits = self.model(imgs)
            xent = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(
                logits=logits, labels=lbls))
        
        grads = tape.gradient(xent, self.weights + self.biases)
        for i in range(n):
            self.weights[i].assign_sub(learning_rate * grads[i])
            self.biases[i].assign_sub(learning_rate * grads[n+i])
        
        return xent

In [9]:
class graph_MLP:
    
    def __init__(self, layer_sizes, init_val):
        if len(layer_sizes) <= 1:
            raise AssertionError("There has to be at least 2 layers. 'layer_sizes' was {}".format(layer_sizes))
        
        self.weights = [tf.Variable(np.random.uniform(low=-init_val, high=init_val, size=(layer_sizes[i], layer_sizes[i+1])).astype(np.float32)) for i in range(len(layer_sizes)-1)]
        self.biases = [tf.Variable(np.zeros(layer_sizes[i]).astype(np.float32)) for i in range(1,len(layer_sizes))]
    
    def model(self, input_batch):
        """returns the logits for a given input batch"""
        
        n = len(self.biases)
        if n == 1:
            return tf.matmul(input_batch, self.weights[0]) + self.biases[0]
        
        next_input = input_batch
        for i in range(n-1):
            next_input = tf.nn.leaky_relu(tf.matmul(next_input, self.weights[i]) + self.biases[i])
        
        return tf.matmul(next_input, self.weights[n-1]) + self.biases[n-1]
    
    @tf.function
    def train(self, imgs, lbls, learning_rate):
        """returns the cross entropy error"""
        n = len(self.biases)
      
        with tf.GradientTape() as tape:
            logits = self.model(imgs)
            xent = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(
                logits=logits, labels=lbls))
        
        grads = tape.gradient(xent, self.weights + self.biases)
        for i in range(n):
            self.weights[i].assign_sub(learning_rate * grads[i])
            self.biases[i].assign_sub(learning_rate * grads[n+i])
        
        return xent
            
            

In [12]:
(train_imgs, train_lbls), (test_imgs, test_lbls) = tf.keras.datasets.mnist.load_data()

train_data = tf.data.Dataset.from_tensor_slices(
    (train_imgs.reshape((-1, 784)).astype(np.float32) / 255, train_lbls.astype(np.int32))).shuffle(buffer_size=60000).batch(128).repeat()

test_data = tf.data.Dataset.from_tensor_slices(
    (test_imgs.reshape((-1, 784)).astype(np.float32) / 255, test_lbls.astype(np.int32))).batch(128)


In [14]:
mlp_times = []
for _ in range(10):
    start = datetime.now()
    mlp = MLP([784, 200, 10], 0.1)
    for i, (imgs, lbls) in enumerate(train_data):
        if i > 2000:
            break
        mlp.train(imgs, lbls, 0.1)
    mlp_times.append(datetime.now() - start)

g_mlp_times = []
for _ in range(10):
    start = datetime.now()
    g_mlp = graph_MLP([784, 200, 10], 0.1)
    for i, (imgs, lbls) in enumerate(train_data):
        if i > 2000:
            break
        g_mlp.train(imgs, lbls, 0.1)
    g_mlp_times.append(datetime.now() - start)
    
print("eager: mean = {}".format(np.mean(mlp_times)))
print("graph: mean = {}".format(np.mean(g_mlp_times)))

eager: mean = 0:00:08.766914
graph: mean = 0:00:04.668776


## ResNet